Predicts player performance using Linear Regression and Ridge Regression

Also has helpers to fix MP column and splitting d

# Installing/Importing Necessary Libraries

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# stop the pandas indexing/splicing warning from appearing
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# df1 = pd.read_csv('dataset_2019-20.csv')
# df = pd.read_csv('2019-20_splitpositions.csv')
# df1 = pd.read_csv('dataset_2019-20_addedFeatures2.csv')
# df1 = pd.read_csv('dataset_2020-21.csv')
# df1 = pd.read_csv('/content/dataset_2020-21_seasonAvgs.csv')
# df1 = pd.read_csv('/content/reviseddataset_2020-21_addedFeatures3.csv')
df1 = pd.read_csv('/content/2020-21_seasonAvgAndRidgeReg.csv')
df1.sort_values(['Date', 'Salary', 'FPTS'], ascending=[True, True, False], inplace=True)
# df = df[['Date', 'Name', 'Team', 'Position', 'Salary', 'FPTS', 'RollingAvg3', 'RollingAvg5', 'RollingAvg7', 'RollingAvg10', 'SeasonAvg']]
# df = df[df.FPTS >= 0] # drop all players who scored less than 0 points
df1.shape

(21797, 40)

In [ ]:
df2 = pd.read_csv('/content/2020-21_seasonAvgPredictions.csv')
df2.sort_values(['Date', 'Salary', 'FPTS'], ascending=[True, True, False], inplace=True)
df2.dropna(subset=['Season_PTS', 'Season_TRB', 'Season_3P', 'Season_AST', 'Season_STL', 'Season_BLK', 'Season_TOV', 'Season_DD', 'Season_TD'], inplace=True)

# df = df[['Date', 'Name', 'Team', 'Position', 'Salary', 'FPTS', 'RollingAvg3', 'RollingAvg5', 'RollingAvg7', 'RollingAvg10', 'SeasonAvg']]
# df = df[df.FPTS >= 0] # drop all players who scored less than 0 points
df2.shape

(21797, 39)

In [ ]:
# all the unique dates in the dataset
dates = df1['Date'].unique()

In [ ]:
df1

,Date,Name,Team,Starter,Position,Salary,FPTS,MP,FG,FGA,...,Season_TRB,Season_3P,Season_AST,Season_STL,Season_BLK,Season_TOV,Season_DD,Season_TD,SeasonAvgPred,RidgeRegPred
0,20201225,Precious Achiuwa,Mia,0,PF/C,3000,19.25,19.47,5,7,...,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.750000,12.596637
1,20201225,Sam Merrill,Mil,0,SG,3000,18.25,7.62,2,3,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.538138
2,20201225,Payton Pritchard,Bos,0,PG,3000,14.00,25.10,3,5,...,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,6.750000,7.686159
3,20201225,Bryn Forbes,Mil,0,PG/SG,3000,9.00,13.35,3,5,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.750000,6.295564
4,20201225,Willie Cauley-Stein,Dal,0,C,3000,8.00,14.08,0,3,...,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,7.412774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21792,20210701,Kevin Huerter,Atl,1,SG/SF,5800,28.25,37.73,3,12,...,3.456790,1.950617,3.308642,1.061728,0.395062,1.086420,0.049383,0.000000,24.444444,23.966851
21793,20210701,Clint Capela,Atl,1,C,6300,23.00,21.13,2,3,...,13.586667,0.000000,0.786667,0.693333,1.826667,1.173333,0.706667,0.013333,37.876667,35.833791
21794,20210701,John Collins,Atl,1,PF/C,6700,35.00,34.13,7,16,...,7.513514,1.270270,1.162162,0.486486,0.864865,1.270270,0.229730,0.000000,30.750000,29.966399
21795,20210701,Jrue Holiday,Mil,1,PG/SG,8300,56.00,41.92,9,20,...,4.593750,1.875000,6.500000,1.609375,0.609375,2.125000,0.187500,0.000000,37.617188,36.677466


In [ ]:
df2

,Date,Name,Team,Starter,Position,Salary,FPTS,MP,FG,FGA,...,Season_PTS,Season_TRB,Season_3P,Season_AST,Season_STL,Season_BLK,Season_TOV,Season_DD,Season_TD,SeasonAvgPred
284,20201225,Precious Achiuwa,Mia,0,PF/C,3000,19.25,19.47,5,7,...,8.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.750000
285,20201225,Sam Merrill,Mil,0,SG,3000,18.25,7.62,2,3,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
287,20201225,Payton Pritchard,Bos,0,PG,3000,14.00,25.10,3,5,...,3.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,6.750000
288,20201225,Bryn Forbes,Mil,0,PG/SG,3000,9.00,13.35,3,5,...,3.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.750000
290,20201225,Willie Cauley-Stein,Dal,0,C,3000,8.00,14.08,0,3,...,2.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22304,20210701,Kevin Huerter,Atl,1,SG/SF,5800,28.25,37.73,3,12,...,11.740741,3.456790,1.950617,3.308642,1.061728,0.395062,1.086420,0.049383,0.000000,24.444444
22305,20210701,Clint Capela,Atl,1,C,6300,23.00,21.13,2,3,...,14.160000,13.586667,0.000000,0.786667,0.693333,1.826667,1.173333,0.706667,0.013333,37.876667
22306,20210701,John Collins,Atl,1,PF/C,6700,35.00,34.13,7,16,...,16.567568,7.513514,1.270270,1.162162,0.486486,0.864865,1.270270,0.229730,0.000000,30.750000
22307,20210701,Jrue Holiday,Mil,1,PG/SG,8300,56.00,41.92,9,20,...,17.531250,4.593750,1.875000,6.500000,1.609375,0.609375,2.125000,0.187500,0.000000,37.617188


# Fixing the MP column to correctly represent the minutes played in minutes


In [ ]:
from math import modf

In [ ]:
df = pd.read_csv('/content/dataset2_splitpositions.csv')


In [ ]:
# fixing the minutes to correctly represent the decimal values as 
min_fixed = []
for index, row in df.iterrows():
  # print(row['MP'])
  s, m = modf(row['MP'])
  # print(m,s)
  s = s/0.6
  # print(m,s)

  time = round(m+s,2)
  print(time)

  min_fixed.append(time)

Streaming output truncated to the last 5000 lines.
32.32
32.32
32.32
32.32
0.5
0.5
0.5
38.32
38.32
38.32
24.83
24.83
19.17
19.17
19.17
19.17
4.13
4.13
26.07
26.07
26.07
24.12
24.12
24.12
24.12
16.18
16.18
19.08
19.08
38.8
38.8
38.8
38.8
38.08
38.08
38.08
33.6
33.6
33.6
33.6
33.02
33.02
33.02
33.02
40.98
40.98
40.98
40.98
40.98
22.88
22.88
22.88
22.88
12.67
12.67
12.67
36.17
36.17
36.17
34.57
34.57
20.02
20.02
23.53
23.53
23.53
31.35
31.35
31.35
11.28
11.28
11.28
11.28
27.9
27.9
27.9
22.25
22.25
22.25
13.03
13.03
13.03
13.03
36.72
36.72
34.13
34.13
34.13
22.95
22.95
37.73
37.73
37.73
37.73
18.3
18.3
18.3
18.3
18.3
45.18
45.18
45.18
36.08
36.08
36.08
36.08
36.08
18.23
18.23
2.03
2.03
2.03
2.03
2.03
8.02
8.02
8.02
0.53
0.53
0.53
0.53
4.88
4.88
4.88
4.88
4.88
18.42
18.42
18.42
18.42
16.15
16.15
16.15
22.62
22.62
22.62
22.62
22.62
21.1
21.1
21.1
21.1
35.88
35.88
35.88
29.42
29.42
29.42
29.42
32.8
32.8
32.8
32.8
32.8
29.77
29.77
35.23
35.23
35.23
35.23
39.35
39.35
30.17
30.17
30.17
30.17
36.

In [ ]:
df['MP'] = min_fixed

In [ ]:
df.to_csv(f'dataset_2020-21_fixed.csv', line_terminator='\n', index=False)


In [ ]:
df2 = df[df['Date'] >= 20201222]
df2.sort_values(['Date', 'Salary', 'FPTS'], ascending=[True, True, False], inplace=True, ignore_index=True)
df2

,Date,Name,Team,Starter,Position,Salary,FPTS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+-
0,20201222,Reggie Perry,Bkn,0,C,3000,6.00,7.77,1,3,...,1,1,2,1,0,0,0,0,2,-8
1,20201222,Reggie Perry,Bkn,0,F,3000,6.00,7.77,1,3,...,1,1,2,1,0,0,0,0,2,-8
2,20201222,Reggie Perry,Bkn,0,PF,3000,6.00,7.77,1,3,...,1,1,2,1,0,0,0,0,2,-8
3,20201222,Reggie Perry,Bkn,0,UTIL,3000,6.00,7.77,1,3,...,1,1,2,1,0,0,0,0,2,-8
4,20201222,Mfiondu Kabengele,LAC,0,C,3000,0.00,0.93,0,0,...,0,0,0,0,0,0,0,0,0,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76864,20210701,Khris Middleton,Mil,1,F,9400,62.25,44.97,10,20,...,3,10,13,8,2,1,1,2,26,16
76865,20210701,Khris Middleton,Mil,1,G,9400,62.25,44.97,10,20,...,3,10,13,8,2,1,1,2,26,16
76866,20210701,Khris Middleton,Mil,1,SF,9400,62.25,44.97,10,20,...,3,10,13,8,2,1,1,2,26,16
76867,20210701,Khris Middleton,Mil,1,SG,9400,62.25,44.97,10,20,...,3,10,13,8,2,1,1,2,26,16


In [ ]:
df2.to_csv(f'2020-21_splitpositions_fixed.csv', line_terminator='\n', index=False)

# Splitting The Dataframe

In [ ]:
df = pd.read_csv('/content/2020-21_seasonAvgAndRidgeReg.csv')

In [ ]:
# all the unique dates in the dataset
dates = df['Date'].unique()

In [ ]:
split_df = pd.DataFrame()

for i in range(len(dates)):
  date = dates[i]

  curDay = df[df['Date']==date]

  Utils = curDay.copy()
  Utils['Position'] = 'UTIL'
  onePosition = curDay[ curDay['Position'].str.contains('/')==False ]
  multPositions = curDay[curDay['Position'].str.contains("/")]
  multPositions.reset_index(inplace=True, drop=True)

  pos1 = []
  pos2 = []
  for index, row in multPositions.iterrows():
    playerPos1, playerPos2 = row['Position'].split('/')
    copy1 = row.copy()
    copy1['Position'] = playerPos1
    copy2 = row.copy()
    copy2['Position'] = playerPos2
    # print(copy)
    pos1.append(copy1)
    pos2.append(copy2)
    # print(playerPos1, playerPos2)

  pos1 = pd.DataFrame(pos1)
  pos2 = pd.DataFrame(pos2)
  # print(onePosition.shape, pos1.shape, pos2.shape)

  currentPlayers = pd.concat([onePosition, pos1, pos2], ignore_index=True)

  Gs = currentPlayers[currentPlayers['Position'].isin(['PG', 'SG'])]
  Gs.drop_duplicates(subset='Name', inplace=True)
  Gs['Position'] = 'G'

  Fs = currentPlayers[currentPlayers['Position'].isin(['SF', 'PF'])]
  Fs.drop_duplicates(subset='Name', inplace=True)
  Fs['Position'] = 'F'

  currentPlayers = pd.concat([currentPlayers, Gs, Fs, Utils], ignore_index=True)
  currentPlayers.sort_values(['Name', 'Team', 'Position'], na_position='first', inplace=True, ignore_index=True)

  split_df = pd.concat([split_df, currentPlayers], ignore_index=True)

In [ ]:
split_df

,Date,Name,Team,Starter,Position,Salary,FPTS,MP,FG,FGA,...,Season_TRB,Season_3P,Season_AST,Season_STL,Season_BLK,Season_TOV,Season_DD,Season_TD,SeasonAvgPred,RidgeRegPred
264,20201225,Precious Achiuwa,Mia,0,C,3000,19.25,19.47,5,7,...,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,11.750000,12.596637
265,20201225,Precious Achiuwa,Mia,0,F,3000,19.25,19.47,5,7,...,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,11.750000,12.596637
266,20201225,Precious Achiuwa,Mia,0,PF,3000,19.25,19.47,5,7,...,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,11.750000,12.596637
267,20201225,Precious Achiuwa,Mia,0,UTIL,3000,19.25,19.47,5,7,...,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,11.750000,12.596637
281,20201225,Sam Merrill,Mil,0,G,3000,18.25,7.62,2,3,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,2.538138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75116,20210701,Khris Middleton,Mil,1,F,9400,62.25,44.97,10,20,...,6.068493,2.260274,5.246575,1.068493,0.150685,2.753425,0.164384,0.0,38.236301,37.913684
75117,20210701,Khris Middleton,Mil,1,G,9400,62.25,44.97,10,20,...,6.068493,2.260274,5.246575,1.068493,0.150685,2.753425,0.164384,0.0,38.236301,37.913684
75118,20210701,Khris Middleton,Mil,1,SF,9400,62.25,44.97,10,20,...,6.068493,2.260274,5.246575,1.068493,0.150685,2.753425,0.164384,0.0,38.236301,37.913684
75119,20210701,Khris Middleton,Mil,1,SG,9400,62.25,44.97,10,20,...,6.068493,2.260274,5.246575,1.068493,0.150685,2.753425,0.164384,0.0,38.236301,37.913684


In [ ]:
split_df.sort_values(['Date', 'Salary', 'FPTS'], ascending=[True, True, False], inplace=True)


In [ ]:
split_df.to_csv('/content/2020-21_SPLIT_seasonAvgAndRidgeReg.csv', line_terminator='\n', index=False)


# Helper Functions

In [ ]:
# returns a list of dataframes for each position
def buildPositionDFs(df1, order, sort=['Salary', 'FPTS'], incr=[True, False]):
  # incr determines if the players are each position are sorted by increasing salary
  l1 = []
  for i in order:
    # print(i)
    posDF = df1[df1['Position'] == i]
    posDF.sort_values(sort, ascending=incr, inplace=True)
    posDF.reset_index(inplace=True, drop=True)
    l1.append(posDF)
  return l1

In [ ]:
# checkDuplicates takes in a dict of players, checks for duplicates
def checkDuplicates(players):
  setOfPlayers = set()
  for player in players:
      if player in setOfPlayers:
          return True
      else:
          setOfPlayers.add(player)         
  return False

In [ ]:
def getSingleScore(player, df2, param):
  return df2[df2['Name'] == player].iloc[0][param]

# getScore takes in a dict of players, returns the fantasy points scored by players
def getScore(players, df_list, param):
  total_score = 0
  for i in range(len(players)):
    player_score = getSingleScore(players[i], df_list[i], param)
    # print(player, player_score)
    total_score += player_score
  return total_score

In [ ]:
# checkSalary takes in a dict of players, checks if salary > 50000
# if yes return true, else return false
def checkSalary(players):
  total_salary = getSalary(players)
  if total_salary > 50000:
    return True
  return False

def getSingleSalary(player, df2):
  return df2[df2['Name'] == player].iloc[0].Salary

# getSalary takes in a dict of players, returns the combined salary of all players
def getSalary(players, df_list):
  total_salary = 0
  for i in range(len(players)):
    player_sal = getSingleSalary(players[i], df_list[i])
    total_salary += player_sal
  return total_salary

In [ ]:
def getPlayerSingleGame(df1, date, player, position):
  return df1[(df1['Date'] == date) & (df1['Name'] == player) & (df1['Position'] == position)]
  # return df1[(df1['Date'] == date) & (df1['Name'] == player)]

In [ ]:
def getPlayerDF(df1, player):
  return df1[df1['Name'] == player]

In [ ]:
def getSalScore(players, salaries, scores):
  totalSal, totalScore = 0, 0
  for player in players:
    # print(player)
    totalSal += salaries[player]
    totalScore += scores[player]
  return totalSal, totalScore

# Linear Regression Based on MP, Starter, FPTS/MP

In [ ]:
df1['FPTS/MP'] = df1['FPTS']/df1['MP']
df2['FPTS/MP'] = df2['FPTS']/df2['MP']

In [ ]:
# testing the helper functions
getPlayerDF(df1, 'LeBron James')

,Date,Name,Team,Starter,Position,Salary,FPTS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+-,FPTS/MP
28,20191022,LeBron James,LAL,1,PG/SF,9900,43.25,36.00,7,19,...,9,10,8,1,1,5,3,18,-8,1.201389
492,20191025,LeBron James,LAL,1,PG/SF,9500,59.25,30.57,12,22,...,5,7,10,1,0,1,0,32,17,1.938175
758,20191027,LeBron James,LAL,1,PG/SF,9900,47.50,35.02,7,14,...,5,6,12,1,0,4,0,20,17,1.356368
1040,20191029,LeBron James,LAL,1,PG/SF,9700,37.50,28.32,8,15,...,2,2,8,0,1,6,0,23,2,1.324153
1473,20191101,LeBron James,LAL,1,PG/SF,9900,92.50,42.53,13,23,...,12,12,16,4,1,4,5,39,15,2.174935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21028,20200918,LeBron James,LAL,1,PG/SF,11200,40.50,30.90,6,11,...,5,6,12,0,0,3,2,15,15,1.310680
21064,20200920,LeBron James,LAL,1,PG/SF,11200,49.75,36.42,10,20,...,9,11,4,0,2,6,2,26,-9,1.366008
21082,20200922,LeBron James,LAL,1,PG/SF,11100,69.00,37.25,14,23,...,9,10,11,2,2,6,1,30,2,1.852349
21119,20200924,LeBron James,LAL,1,PG/SF,11000,49.75,38.35,7,18,...,8,9,8,0,0,0,2,26,8,1.297262


In [ ]:
# predicts a players fantasy point output based on:
#   a 3 game average of their minutes played
#   whether or not they started their previous game
#   their season average of fantasy points per minute

def prediction1(df1, player, date, n):
  player_season_df = getPlayerDF(df1, player)
  player_df = player_season_df[player_season_df['Date'] < date].tail(n)

  if player_df.shape[0] < n:
    # print(f'not enough games: {player_df.shape[0]}, {n}')
    return np.NAN
  
  start = player_df['Starter'].iloc[-1]
  mp_avg = player_df['MP'].mean()
  fpt_per_min_avg = player_season_df['FPTS/MP'].mean()

  prediction = round(mp_avg*fpt_per_min_avg, 2)

  x = [mp_avg, start, fpt_per_min_avg]
  # print(mp_avg, start, fpt_per_min_avg)
  # return prediction
  return x

In [ ]:
predictions = []

for index, row in df1.iterrows():
  player_name = row['Name']
  date = row['Date']
  prediction = prediction1(df1, player_name, date, 3)
  predictions.append(prediction)

df1['Prediction1'] = predictions


In [ ]:
X = []
y = []

for index, row in df1.iterrows():
  if (index%10000==0):
    print(index/df1.shape[0])

  player_name = row['Name']
  date = row['Date']
  x = prediction1(df1, player_name, date, 3)
  if np.any(np.isnan(x)):
    continue
  X.append(x)

  y_i = row['FPTS']
  y.append(y_i)

X = np.array(X)
y = np.array(y)

0.0
0.47229962688329474
0.9445992537665895


In [ ]:
X, X.shape, y, y.shape

(array([[13.73333333,  0.        ,  0.7229394 ],
        [ 9.20666667,  0.        ,  0.54732814],
        [10.39666667,  0.        ,  1.0491169 ],
        ...,
        [37.45      ,  1.        ,  1.19863154],
        [38.99      ,  1.        ,  1.22563163],
        [41.31      ,  1.        ,  1.21664908]]),
 (19697, 3),
 array([ 2.75, 24.5 , 15.75, ..., 38.  , 59.5 , 57.25]),
 (19697,))

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
reg = linear_model.LinearRegression()
reg.fit(X, y)

LinearRegression()

In [ ]:
X_test = []
y_test = []

for index, row in df2.iterrows():
  if (index%10000==0):
    print(index/df2.shape[0])

  player_name = row['Name']
  date = row['Date']
  x = prediction1(df2, player_name, date, 3)
  if np.any(np.isnan(x)):
    continue
  X_test.append(x)

  y_i = row['FPTS']
  y_test.append(y_i)

X_test = np.array(X_test)
y_test = np.array(y_test)

0.0
0.4482495853691335
0.896499170738267


In [ ]:
reg.coef_

array([ 0.66960275,  1.92063682, 26.90115967])

In [ ]:
y_pred = reg.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_pred)

96.32092849015977

In [ ]:
r2_score(y_test, y_pred)

0.5785482763390584

# Linear Regression Based on Average of Last N Games

In [ ]:
# predicts a players fantasy point output based on:
#   the average of their counting stats over the last n games

def prediction2(df1, player, date, n):
  player_season_df = getPlayerDF(df1, player)
  player_df = player_season_df[player_season_df['Date'] < date].tail(n)

  if player_df.shape[0] < n:
    # print(f'not enough games: {player_df.shape[0]}, {n}')
    return np.NAN

  mp_avg = player_df['MP'].mean()
  start = player_df['Starter'].iloc[-1]
  pts_avg = player_df['PTS'].mean()
  reb_avg = player_df['TRB'].mean()
  thr_avg = player_df['3P'].mean()
  ast_avg = player_df['AST'].mean()
  stl_avg = player_df['STL'].mean()
  blk_avg = player_df['BLK'].mean()
  tov_avg = player_df['TOV'].mean()


  x = [mp_avg, start, pts_avg, reb_avg, thr_avg, ast_avg, stl_avg, blk_avg, tov_avg]

  return x

In [ ]:
X = []
y = []

for index, row in df1.iterrows():
  if (index%1000==0):
    print(index/df1.shape[0])

  player_name = row['Name']
  date = row['Date']
  x = prediction2(df1, player_name, date, 3)
  if np.any(np.isnan(x)):
    continue
  X.append(x)

  y_i = row['FPTS']
  y.append(y_i)

X = np.array(X)
y = np.array(y)

0.0
0.04722996268832948
0.09445992537665895
0.14168988806498842
0.1889198507533179
0.23614981344164737
0.28337977612997683
0.33060973881830635
0.3778397015066358
0.4250696641949653
0.47229962688329474
0.5195295895716242
0.5667595522599537
0.6139895149482832
0.6612194776366127
0.7084494403249422
0.7556794030132716
0.8029093657016011
0.8501393283899306
0.89736929107826
0.9445992537665895
0.991829216454919


In [ ]:
reg = linear_model.LinearRegression()
reg.fit(X, y)

LinearRegression()

In [ ]:
X_test = []
y_test = []

for index, row in df2.iterrows():
  if (index%10000==0):
    print(index/df2.shape[0])

  player_name = row['Name']
  date = row['Date']
  x = prediction2(df2, player_name, date, 3)
  if np.any(np.isnan(x)):
    continue
  X_test.append(x)

  y_i = row['FPTS']
  y_test.append(y_i)

X_test = np.array(X_test)
y_test = np.array(y_test)

0.0
0.4482495853691335
0.896499170738267


In [ ]:
y_pred = reg.predict(X_test)

In [ ]:
y_pred[y_pred < 0]

array([], dtype=float64)

In [ ]:
mean_squared_error(y_test, y_pred)


100.90968517267642

In [ ]:
r2_score(y_test, y_pred)

0.5584701952447209

# Reproducing Barry, Canova, Capiz

In [ ]:
scoring = {'PTS':1, '3P':0.5, 'TRB':1.25, 'AST':1.5, 'STL':2, 'BLK':2, 'TOV':-.5, 'DD':1.5, 'TD':3}

**Adding Double-Double, Triple-Double Columns**

In [ ]:
df1

,Date,Name,Team,Starter,Position,Salary,FPTS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+-
0,20191022,Nicolo Melli,Nor,0,PF/C,3000,24.25,19.37,5,7,...,2,3,5,2,0,0,2,1,14,11
1,20191022,Kenrich Williams,Nor,0,SF/PF,3100,20.50,18.02,0,4,...,3,3,6,3,1,2,1,5,3,11
2,20191022,Jared Dudley,LAL,0,SF/PF,3100,7.00,13.21,2,2,...,0,0,0,0,0,0,0,2,6,-20
3,20191022,Josh Hart,Nor,0,SG/SF,3200,33.50,28.10,4,9,...,4,6,10,1,0,1,1,4,15,-1
4,20191022,Jahlil Okafor,Nor,0,C,3200,12.00,12.29,3,3,...,2,0,2,0,0,1,1,3,8,-7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21168,20200927,Kemba Walker,Bos,1,PG,7500,31.00,35.00,7,15,...,0,2,2,5,0,0,1,5,20,-3
21169,20200927,Jaylen Brown,Bos,1,SG/SF,7800,48.50,40.22,10,17,...,2,6,8,4,3,0,2,0,26,-17
21170,20200927,Jimmy Butler,Mia,1,SG/SF,8100,38.00,39.01,8,19,...,0,2,2,8,1,0,1,2,22,18
21171,20200927,Bam Adebayo,Mia,1,C,8400,59.50,39.29,11,15,...,1,13,14,5,1,0,2,3,32,0


In [ ]:
DD = []
TD = []

for index, row in df1.iterrows():
  count = 0
  for cat in scoring.keys():
    if cat not in ['3P', 'TOV', 'DD', 'TD']:
      if row[cat] >= 10:
        count += 1
  
  if count >= 2:
    DD.append(1)
  else: 
    DD.append(0)

  if count >= 3:
    TD.append(1)
  else: 
    TD.append(0)

In [ ]:
df1['DD'] = DD
df1['TD'] = TD

In [ ]:
df1

,Date,Name,Team,Starter,Position,Salary,FPTS,MP,FG,FGA,...,TRB,AST,STL,BLK,TOV,PF,PTS,+-,DD,TD
0,20191022,Nicolo Melli,Nor,0,PF/C,3000,24.25,19.37,5,7,...,5,2,0,0,2,1,14,11,0,0
1,20191022,Kenrich Williams,Nor,0,SF/PF,3100,20.50,18.02,0,4,...,6,3,1,2,1,5,3,11,0,0
2,20191022,Jared Dudley,LAL,0,SF/PF,3100,7.00,13.21,2,2,...,0,0,0,0,0,2,6,-20,0,0
3,20191022,Josh Hart,Nor,0,SG/SF,3200,33.50,28.10,4,9,...,10,1,0,1,1,4,15,-1,1,0
4,20191022,Jahlil Okafor,Nor,0,C,3200,12.00,12.29,3,3,...,2,0,0,1,1,3,8,-7,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21168,20200927,Kemba Walker,Bos,1,PG,7500,31.00,35.00,7,15,...,2,5,0,0,1,5,20,-3,0,0
21169,20200927,Jaylen Brown,Bos,1,SG/SF,7800,48.50,40.22,10,17,...,8,4,3,0,2,0,26,-17,0,0
21170,20200927,Jimmy Butler,Mia,1,SG/SF,8100,38.00,39.01,8,19,...,2,8,1,0,1,2,22,18,0,0
21171,20200927,Bam Adebayo,Mia,1,C,8400,59.50,39.29,11,15,...,14,5,1,0,2,3,32,0,1,0


In [ ]:
df1.to_csv(f'dataset_2020-21_addedFeatures.csv', line_terminator='\n', index=False)

**Adding Season Average Feature Columns**

In [ ]:
df1.columns

Index(['Date', 'Name', 'Team', 'Starter', 'Position', 'Salary', 'FPTS', 'MP',
       'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+-', 'DD',
       'TD'],
      dtype='object')

In [ ]:
def getFeatureSeasonAvg(player_df, feature):
  if player_df.shape[0] == 0:
    # print(f'no games: {player_df.shape[0]}, {player}')
    return np.NAN

  avg = player_df[feature].mean()
  return avg

In [ ]:
features = ['PTS', 'TRB', '3P', 'AST', 'STL', 'BLK', 'TOV', 'DD', 'TD']

In [ ]:
szn_pts = []
szn_trb = []
szn_3p = []
szn_ast = []
szn_stl = []
szn_blk = []
szn_tov = []
szn_dd = []
szn_td = []
for index, row in df1.iterrows():
  name = row['Name']
  date = row['Date']
  player_df = getPlayerDF(df1, name)
  player_df = player_df[player_df['Date'] < date]

  szn_pts.append(getFeatureSeasonAvg(player_df, 'PTS'))
  szn_trb.append(getFeatureSeasonAvg(player_df, 'TRB'))
  szn_3p.append(getFeatureSeasonAvg(player_df, '3P'))
  szn_ast.append(getFeatureSeasonAvg(player_df, 'AST'))
  szn_stl.append(getFeatureSeasonAvg(player_df, 'STL'))
  szn_blk.append(getFeatureSeasonAvg(player_df, 'BLK'))
  szn_tov.append(getFeatureSeasonAvg(player_df, 'TOV'))
  szn_dd.append(getFeatureSeasonAvg(player_df, 'DD'))
  szn_td.append(getFeatureSeasonAvg(player_df, 'TD'))

In [ ]:
df1['Season_PTS'] = szn_pts
df1['Season_TRB'] = szn_trb
df1['Season_3P'] = szn_3p
df1['Season_AST'] = szn_ast
df1['Season_STL'] = szn_stl
df1['Season_BLK'] = szn_blk
df1['Season_TOV'] = szn_tov
df1['Season_DD'] = szn_dd
df1['Season_TD'] = szn_td

In [ ]:
df1.columns

Index(['Date', 'Name', 'Team', 'Starter', 'Position', 'Salary', 'FPTS', 'MP',
       'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+-', 'DD', 'TD',
       'Season_PTS', 'Season_TRB', 'Season_3P', 'Season_AST', 'Season_STL',
       'Season_BLK', 'Season_TOV', 'Season_DD', 'Season_TD'],
      dtype='object')

In [ ]:
df1

,Date,Name,Team,Starter,Position,Salary,FPTS,MP,FG,FGA,...,TD,Season_PTS,Season_TRB,Season_3P,Season_AST,Season_STL,Season_BLK,Season_TOV,Season_DD,Season_TD
0,20191022,Nicolo Melli,Nor,0,PF/C,3000,24.25,19.37,5,7,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20191022,Kenrich Williams,Nor,0,SF/PF,3100,20.50,18.02,0,4,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20191022,Jared Dudley,LAL,0,SF/PF,3100,7.00,13.21,2,2,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20191022,Josh Hart,Nor,0,SG/SF,3200,33.50,28.10,4,9,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20191022,Jahlil Okafor,Nor,0,C,3200,12.00,12.29,3,3,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21168,20200927,Kemba Walker,Bos,1,PG,7500,31.00,35.00,7,15,...,0,20.761905,3.873016,3.111111,4.777778,0.984127,0.492063,2.063492,0.047619,0.000000
21169,20200927,Jaylen Brown,Bos,1,SG/SF,7800,48.50,40.22,10,17,...,0,20.365079,6.492063,2.253968,2.095238,1.190476,0.412698,2.285714,0.126984,0.000000
21170,20200927,Jimmy Butler,Mia,1,SG/SF,8100,38.00,39.01,8,19,...,0,20.092308,6.400000,0.600000,5.553846,1.846154,0.523077,2.292308,0.153846,0.030769
21171,20200927,Bam Adebayo,Mia,1,C,8400,59.50,39.29,11,15,...,0,16.051948,10.363636,0.025974,5.012987,1.168831,1.285714,2.779221,0.558442,0.025974


In [ ]:
df1.to_csv(f'dataset_2019-20_addedFeatures2.csv', line_terminator='\n', index=False)


**Basic Estimation**

Using season averages (up to the current game) to predict current game

In [ ]:
df1.columns


Index(['Date', 'Name', 'Team', 'Starter', 'Position', 'Salary', 'FPTS', 'MP',
       'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+-', 'DD', 'TD',
       'Season_PTS', 'Season_TRB', 'Season_3P', 'Season_AST', 'Season_STL',
       'Season_BLK', 'Season_TOV', 'Season_DD', 'Season_TD', 'Pred1',
       'RidgeRegPred'],
      dtype='object')

In [ ]:
# predicts a players fantasy point output based on:
#   the average of their counting stats over the last n games

def basicEstimation(df1, player, date):
  player_season_df = getPlayerDF(df1, player)
  player_df = player_season_df[player_season_df['Date'] <= date]

  if player_df.shape[0] == 1:
    # print(f'no games: {player_df.shape[0]}, {player}')
    return np.NAN

  avg = {}

  # print(player_df['Season_PTS'].iloc[-1])
  # print(player_df['Season_PTS'])
  avg['PTS'] = player_df['Season_PTS'].iloc[-1]
  avg['TRB'] = player_df['Season_TRB'].iloc[-1]
  avg['3P'] = player_df['Season_3P'].iloc[-1]
  avg['AST'] = player_df['Season_AST'].iloc[-1]
  avg['STL'] = player_df['Season_STL'].iloc[-1]
  avg['BLK'] = player_df['Season_BLK'].iloc[-1]
  avg['TOV'] = player_df['Season_TOV'].iloc[-1]
  avg['DD'] = player_df['Season_DD'].iloc[-1]
  avg['TD'] = player_df['Season_TD'].iloc[-1]

  pred = 0
  for cat in scoring:
    pred += scoring[cat]*avg[cat]
    print(f'{cat}\t{avg[cat]}*{scoring[cat]} = {scoring[cat]*avg[cat]}')
    # print(f'{scoring[cat]}')
  return pred

In [ ]:
basicEstimates = []
for index, row in df1.iterrows():
  name = row['Name']
  date = row['Date']
  pred = basicEstimation(df1, name, date)
  print(f'INDEX: {index}\tNAME: {name}\tDATE: {date}\tPrediction: {pred}')
  
  basicEstimates.append(pred)


In [ ]:
df1['SeasonAvgPred'] = basicEstimates


In [ ]:
df1.shape

(21797, 41)

In [ ]:
df1.to_csv('2020-21_seasonAvgPredictions.csv', line_terminator='\n', index=False)


**Calculating Root Mean Squared Error**

2019-2020: 9.96

2020-2021: 9.94

In [ ]:
df1

,Date,Name,Team,Starter,Position,Salary,FPTS,MP,FG,FGA,...,Season_PTS,Season_TRB,Season_3P,Season_AST,Season_STL,Season_BLK,Season_TOV,Season_DD,Season_TD,SeasonAvgPred
0,20201222,Reggie Perry,Bkn,0,PF/C,3000,6.00,7.77,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20201222,Mfiondu Kabengele,LAC,0,PF/C,3000,0.00,0.93,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20201222,Amir Coffey,LAC,0,SF,3100,0.00,0.93,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20201222,Quinn Cook,LAL,0,PG,3200,0.00,1.38,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20201222,Brad Wanamaker,GSW,0,PG,3300,8.25,21.67,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22304,20210701,Kevin Huerter,Atl,1,SG/SF,5800,28.25,37.73,3,12,...,11.740741,3.456790,1.950617,3.308642,1.061728,0.395062,1.086420,0.049383,0.000000,24.444444
22305,20210701,Clint Capela,Atl,1,C,6300,23.00,21.13,2,3,...,14.160000,13.586667,0.000000,0.786667,0.693333,1.826667,1.173333,0.706667,0.013333,37.876667
22306,20210701,John Collins,Atl,1,PF/C,6700,35.00,34.13,7,16,...,16.567568,7.513514,1.270270,1.162162,0.486486,0.864865,1.270270,0.229730,0.000000,30.750000
22307,20210701,Jrue Holiday,Mil,1,PG/SG,8300,56.00,41.92,9,20,...,17.531250,4.593750,1.875000,6.500000,1.609375,0.609375,2.125000,0.187500,0.000000,37.617188


In [ ]:
from sklearn.metrics import mean_squared_error
import math

In [ ]:
# need to ignore NaNs
df1 = df1.dropna(subset=['SeasonAvgPred'])
# df2 = df2.dropna(subset=['Pred1'])

In [ ]:
df1

,Date,Name,Team,Starter,Position,Salary,FPTS,MP,FG,FGA,...,Season_PTS,Season_TRB,Season_3P,Season_AST,Season_STL,Season_BLK,Season_TOV,Season_DD,Season_TD,SeasonAvgPred
284,20201225,Precious Achiuwa,Mia,0,PF/C,3000,19.25,19.47,5,7,...,8.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.750000
285,20201225,Sam Merrill,Mil,0,SG,3000,18.25,7.62,2,3,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
287,20201225,Payton Pritchard,Bos,0,PG,3000,14.00,25.10,3,5,...,3.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,6.750000
288,20201225,Bryn Forbes,Mil,0,PG/SG,3000,9.00,13.35,3,5,...,3.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.750000
290,20201225,Willie Cauley-Stein,Dal,0,C,3000,8.00,14.08,0,3,...,2.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22304,20210701,Kevin Huerter,Atl,1,SG/SF,5800,28.25,37.73,3,12,...,11.740741,3.456790,1.950617,3.308642,1.061728,0.395062,1.086420,0.049383,0.000000,24.444444
22305,20210701,Clint Capela,Atl,1,C,6300,23.00,21.13,2,3,...,14.160000,13.586667,0.000000,0.786667,0.693333,1.826667,1.173333,0.706667,0.013333,37.876667
22306,20210701,John Collins,Atl,1,PF/C,6700,35.00,34.13,7,16,...,16.567568,7.513514,1.270270,1.162162,0.486486,0.864865,1.270270,0.229730,0.000000,30.750000
22307,20210701,Jrue Holiday,Mil,1,PG/SG,8300,56.00,41.92,9,20,...,17.531250,4.593750,1.875000,6.500000,1.609375,0.609375,2.125000,0.187500,0.000000,37.617188


In [ ]:
meansqre_err = mean_squared_error(df1['FPTS'], df1['SeasonAvgPred'])
# meansqre_err = mean_squared_error(df2['FPTS'], df2['Pred1'])

In [ ]:
root_meansqre_err = math.sqrt(meansqre_err)

In [ ]:
root_meansqre_err

9.940774066804233

**Calculating Mean Absolute Error**

2019-2020: 7.75

2020-2021: 7.70

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
mean_abs_err = mean_absolute_error(df1['FPTS'], df1['SeasonAvgPred'])
# mean_abs_err = mean_absolute_error(df2['FPTS'], df2['Pred1'])

In [ ]:
mean_abs_err

7.695624712104944

**Ridge Regression (Part G on the paper)**

Avoid overfitting by applying a penalty, improves models ability to generalize

Using 10-fold cross validation, lambda that minimizes penalty MSE

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold, cross_val_score, cross_validate
from sklearn.linear_model import RidgeCV

In [ ]:
df1

,Date,Name,Team,Starter,Position,Salary,FPTS,MP,FG,FGA,...,TD,Season_PTS,Season_TRB,Season_3P,Season_AST,Season_STL,Season_BLK,Season_TOV,Season_DD,Season_TD
0,20191022,Nicolo Melli,Nor,0,PF/C,3000,24.25,19.37,5,7,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20191022,Kenrich Williams,Nor,0,SF/PF,3100,20.50,18.02,0,4,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20191022,Jared Dudley,LAL,0,SF/PF,3100,7.00,13.21,2,2,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20191022,Josh Hart,Nor,0,SG/SF,3200,33.50,28.10,4,9,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20191022,Jahlil Okafor,Nor,0,C,3200,12.00,12.29,3,3,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21168,20200927,Kemba Walker,Bos,1,PG,7500,31.00,35.00,7,15,...,0,20.761905,3.873016,3.111111,4.777778,0.984127,0.492063,2.063492,0.047619,0.000000
21169,20200927,Jaylen Brown,Bos,1,SG/SF,7800,48.50,40.22,10,17,...,0,20.365079,6.492063,2.253968,2.095238,1.190476,0.412698,2.285714,0.126984,0.000000
21170,20200927,Jimmy Butler,Mia,1,SG/SF,8100,38.00,39.01,8,19,...,0,20.092308,6.400000,0.600000,5.553846,1.846154,0.523077,2.292308,0.153846,0.030769
21171,20200927,Bam Adebayo,Mia,1,C,8400,59.50,39.29,11,15,...,0,16.051948,10.363636,0.025974,5.012987,1.168831,1.285714,2.779221,0.558442,0.025974


In [ ]:
df1.columns

Index(['Date', 'Name', 'Team', 'Starter', 'Position', 'Salary', 'FPTS', 'MP',
       'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+-', 'DD', 'TD',
       'Season_PTS', 'Season_TRB', 'Season_3P', 'Season_AST', 'Season_STL',
       'Season_BLK', 'Season_TOV', 'Season_DD', 'Season_TD'],
      dtype='object')

In [ ]:
df1.dropna(subset=['Season_PTS', 'Season_TRB', 'Season_3P', 'Season_AST', 'Season_STL', 'Season_BLK', 'Season_TOV', 'Season_DD', 'Season_TD'], inplace=True)
# df1.dropna(subset=['Season_PTS', 'Season_TRB', 'Season_3P', 'Season_AST', 'Season_STL', 'Season_BLK', 'Season_TOV'], inplace=True)

In [ ]:
df1

,Date,Name,Team,Starter,Position,Salary,FPTS,MP,FG,FGA,...,TD,Season_PTS,Season_TRB,Season_3P,Season_AST,Season_STL,Season_BLK,Season_TOV,Season_DD,Season_TD
247,20191024,Tim Frazier,Det,0,PG,3000,9.00,10.13,2,5,...,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
254,20191024,Patrick Patterson,LAC,1,PF/C,3100,27.75,28.10,7,13,...,0,4.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
255,20191024,Langston Galloway,Det,0,PG/SG,3100,20.75,25.51,2,9,...,0,5.000000,4.000000,1.000000,0.000000,1.000000,1.000000,3.000000,0.000000,0.000000
273,20191024,Thon Maker,Det,0,PF/C,3300,12.25,19.16,0,1,...,0,4.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
275,20191024,Tony Snell,Det,1,SG/SF,3400,18.00,33.36,3,8,...,0,10.000000,2.000000,2.000000,2.000000,0.000000,0.000000,1.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21168,20200927,Kemba Walker,Bos,1,PG,7500,31.00,35.00,7,15,...,0,20.761905,3.873016,3.111111,4.777778,0.984127,0.492063,2.063492,0.047619,0.000000
21169,20200927,Jaylen Brown,Bos,1,SG/SF,7800,48.50,40.22,10,17,...,0,20.365079,6.492063,2.253968,2.095238,1.190476,0.412698,2.285714,0.126984,0.000000
21170,20200927,Jimmy Butler,Mia,1,SG/SF,8100,38.00,39.01,8,19,...,0,20.092308,6.400000,0.600000,5.553846,1.846154,0.523077,2.292308,0.153846,0.030769
21171,20200927,Bam Adebayo,Mia,1,C,8400,59.50,39.29,11,15,...,0,16.051948,10.363636,0.025974,5.012987,1.168831,1.285714,2.779221,0.558442,0.025974


In [ ]:
X = np.array(df1[['Season_PTS', 'Season_TRB', 'Season_3P', 'Season_AST', 'Season_STL', 'Season_BLK', 'Season_TOV', 'Season_DD', 'Season_TD']])
# X = np.array(df1[['Season_PTS', 'Season_TRB', 'Season_3P', 'Season_AST', 'Season_STL', 'Season_BLK', 'Season_TOV', 'DD', 'TD']])
y = np.array(df1['FPTS'])

In [ ]:
# kf = KFold(n_splits=10, shuffle=True, random_state=1)
kf = KFold(n_splits=10, shuffle=False)

for train_index, test_index in kf.split(X):
  print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [ 2068  2069  2070 ... 20672 20673 20674] TEST: [   0    1    2 ... 2065 2066 2067]
TRAIN: [    0     1     2 ... 20672 20673 20674] TEST: [2068 2069 2070 ... 4133 4134 4135]
TRAIN: [    0     1     2 ... 20672 20673 20674] TEST: [4136 4137 4138 ... 6201 6202 6203]
TRAIN: [    0     1     2 ... 20672 20673 20674] TEST: [6204 6205 6206 ... 8269 8270 8271]
TRAIN: [    0     1     2 ... 20672 20673 20674] TEST: [ 8272  8273  8274 ... 10337 10338 10339]
TRAIN: [    0     1     2 ... 20672 20673 20674] TEST: [10340 10341 10342 ... 12404 12405 12406]
TRAIN: [    0     1     2 ... 20672 20673 20674] TEST: [12407 12408 12409 ... 14471 14472 14473]
TRAIN: [    0     1     2 ... 20672 20673 20674] TEST: [14474 14475 14476 ... 16538 16539 16540]
TRAIN: [    0     1     2 ... 20672 20673 20674] TEST: [16541 16542 16543 ... 18605 18606 18607]
TRAIN: [    0     1     2 ... 18605 18606 18607] TEST: [18608 18609 18610 ... 20672 20673 20674]


In [ ]:
ridge = RidgeCV(alphas=[0.1, 0.5, 0.75, 1.0, 1.20 ,1.25])
# ridge = RidgeCV(alphas=[0.1, 1.0, 10.0, ], cv=kf)

In [ ]:
ridge.fit(X, y)

RidgeCV(alphas=array([0.1 , 0.5 , 0.75, 1.  , 1.2 , 1.25]))

In [ ]:
ridge.coef_, ridge.intercept_, ridge.alpha_, ridge.best_score_

(array([0.92521257, 0.88559961, 0.09618895, 1.25301183, 1.39059527,
        1.77021675, 0.78527971, 2.88323422, 1.49784818]),
 2.5381375916787334,
 1.25,
 -97.97575455546685)

In [ ]:
pipeline = make_pipeline(ridge)

In [ ]:
cv_results_rmse = cross_val_score(pipeline, X, y, cv=kf, scoring='neg_root_mean_squared_error', n_jobs=-1)

In [ ]:
cv_results_rmse

array([-10.56154987,  -9.66180223,  -9.23944687,  -9.28463129,
        -9.41331311,  -9.97830141,  -9.84981923, -10.03844465,
       -10.23226851, -10.73043935])

In [ ]:
cv_results_rmse.mean()

-9.89900165188547

In [ ]:
cv_results_mae = cross_val_score(pipeline, X, y, cv=kf, scoring='neg_mean_absolute_error', n_jobs=-1)

In [ ]:
cv_results_mae

array([-8.17255253, -7.57927043, -7.36975209, -7.41212143, -7.43162657,
       -7.82339135, -7.68826243, -7.95069766, -7.96075331, -8.43009528])

In [ ]:
cv_results_mae.mean()

-7.091634567491949

**Predicting on 2020-21 Season Using Fitted Ridge Regression Model**

RMSE: 9.87

MAE: 7.72

In [ ]:
ridge.coef_, ridge.intercept_, ridge.alpha_, ridge.best_score_

(array([0.92521257, 0.88559961, 0.09618895, 1.25301183, 1.39059527,
        1.77021675, 0.78527971, 2.88323422, 1.49784818]),
 2.5381375916787334,
 1.25,
 -97.97575455546685)

In [ ]:
df2

,Date,Name,Team,Starter,Position,Salary,FPTS,MP,FG,FGA,...,Season_TRB,Season_3P,Season_AST,Season_STL,Season_BLK,Season_TOV,Season_DD,Season_TD,SeasonAvgPred,RidgeRegPred
284,20201225,Precious Achiuwa,Mia,0,PF/C,3000,19.25,19.47,5,7,...,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.750000,12.596637
285,20201225,Sam Merrill,Mil,0,SG,3000,18.25,7.62,2,3,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.538138
287,20201225,Payton Pritchard,Bos,0,PG,3000,14.00,25.10,3,5,...,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,6.750000,7.686159
288,20201225,Bryn Forbes,Mil,0,PG/SG,3000,9.00,13.35,3,5,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.750000,6.295564
290,20201225,Willie Cauley-Stein,Dal,0,C,3000,8.00,14.08,0,3,...,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,7.412774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22304,20210701,Kevin Huerter,Atl,1,SG/SF,5800,28.25,37.73,3,12,...,3.456790,1.950617,3.308642,1.061728,0.395062,1.086420,0.049383,0.000000,24.444444,23.966851
22305,20210701,Clint Capela,Atl,1,C,6300,23.00,21.13,2,3,...,13.586667,0.000000,0.786667,0.693333,1.826667,1.173333,0.706667,0.013333,37.876667,35.833791
22306,20210701,John Collins,Atl,1,PF/C,6700,35.00,34.13,7,16,...,7.513514,1.270270,1.162162,0.486486,0.864865,1.270270,0.229730,0.000000,30.750000,29.966399
22307,20210701,Jrue Holiday,Mil,1,PG/SG,8300,56.00,41.92,9,20,...,4.593750,1.875000,6.500000,1.609375,0.609375,2.125000,0.187500,0.000000,37.617188,36.677466


In [ ]:
df2.dropna(subset=['Season_PTS', 'Season_TRB', 'Season_3P', 'Season_AST', 'Season_STL', 'Season_BLK', 'Season_TOV', 'Season_DD', 'Season_TD'], inplace=True)
# df2.dropna(subset=['Season_PTS', 'Season_TRB', 'Season_3P', 'Season_AST', 'Season_STL', 'Season_BLK', 'Season_TOV', 'DD', 'TD'], inplace=True)

X_test = np.array(df2[['Season_PTS', 'Season_TRB', 'Season_3P', 'Season_AST', 'Season_STL', 'Season_BLK', 'Season_TOV', 'Season_DD', 'Season_TD']])
# X_test = np.array(df2[['Season_PTS', 'Season_TRB', 'Season_3P', 'Season_AST', 'Season_STL', 'Season_BLK', 'Season_TOV', 'DD', 'TD']])
y_test = np.array(df2['FPTS'])

In [ ]:
y_pred = ridge.predict(X_test)

In [ ]:
df2['RidgeRegPred'] = y_pred

In [ ]:
mean_abs_err = mean_absolute_error(df2['FPTS'], df2['RidgeRegPred'])
meansqre_err = mean_squared_error(df2['FPTS'], df2['RidgeRegPred'])
root_meansqre_err = math.sqrt(meansqre_err)

In [ ]:
root_meansqre_err, mean_abs_err

(9.868002374537964, 7.725324139709211)

In [ ]:
df2.to_csv(f'reviseddataset_2020-21_seasonAvgRidgeReg.csv', line_terminator='\n', index=False)
